In [ ]:
# TODO pull in the asset universe
# TODO see if i want the asset info on the assets for anything useful; otherwise ignore
# TODO see if anything useful about exchanges; maybe get total volume if they have somewhere
# -remove the function from the OG script?


# TODO pull over needed functions as a copy

# TODO pull all the markets with the pullMarketInfo function; adjust to work for both freq; specificied assets as well
# TODO pull USDT and USDC exchange rates; adjust to work for both freq
# TODO pull market data function but need to adjust to work either for daily level or hourly level;  specificied assets as well
# -pull all markets in those three stables for all my assets to get prices, volume, and trades at HOURLY freq
# --so check first if the market offers hourly freq?
# TODO build the panel


In [ ]:
# TODO figure out how to pully hourly bid and ask for those markets to add to the panel as well

url = 'https://rest.coinapi.io/v1/quotes/BITSTAMP_SPOT_BTC_USD/history?time_start=2016-01-01T00:00:00'


time_start = 1
time_end = 2
limit = 50000

In [ ]:

# TODO move functions to the helper function call

# TODO adjust the OG script to see if they work for it too

